In [9]:
import cobra
from cobra.io import load_model, read_sbml_model

import pandas as pd
import networkx as nx
import os
import glob
import natsort

In [10]:
model_dir = os.path.join(os.getcwd(), "models")
files = natsort.natsorted(glob.glob(model_dir + '/*.xml'))
files

['c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Actinomyces_odontolyticus_ATCC_17982.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Alistipes_putredinis_DSM_17216.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Anaerococcus_hydrogenalis_DSM_7454.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Anaerofustis_stercorihominis_DSM_17244.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Anaerostipes_caccae_DSM_14662.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Anaerotruncus_colihominis_DSM_17241.xml',
 'c:\\Users\\sainz\\OneDrive\\Desktop\\Hamburg 3.2\\_MetabolicSetTheory_repo\\_ModelsAndNetworks\\models\\Bacteroides_ca

In [6]:
for file in files:
    model = read_sbml_model(file)
    species = file.split('\\')[-1].split('.')[0]

    li = []
    for reac in model.reactions:
        li.append(reac.reaction)

    li_id = []
    for reac in model.reactions:
        li_id.append(reac.id)

    # Export possible exchange substances
    ex_index = [i for i in range(len(li_id)) if 'EX_' in li_id[i]]
    possdf = pd.DataFrame([li[i].split(' <=>')[0] for i in ex_index], columns= ['PossibleSubstance'])
    possdf.to_csv(f'possible_substances_{species}.txt', index=None)

    # Export bimass reaction
    biomass_index = [i for i in range(len(li_id)) if li_id[i] == f'{[j for j in li_id if "biomass" in j][-1]}']
    biomass_react = li[biomass_index[0]].split( ' --> ')[0]
    biomass_reactants = [i.split(' ')[-1] for i in biomass_react.split(' + ')]
    biomass_prod = li[biomass_index[0]].split(' --> ')[1]
    biomass_products = [i.split(' ')[-1] for i in biomass_prod.split(' + ')]
    forward_biomass_reaction = (biomass_reactants, biomass_products)

    with open(f'{species}_biomassReaction.txt', 'w') as outfile:
        outfile.write('\n'.join(str(i) for i in forward_biomass_reaction))

    # Segregate all reactions
    forwardreactions_index = [i for i in range(len(li)) if '-->' in li[i]]
    backwardreactions_index = [i for i in range(len(li)) if '<--' in li[i]]
    reversiblereactions_index = [i for i in range(len(li)) if '<=>' in li[i]]

    forwardreactions = [li[i] for i in range(len(li)) if i in forwardreactions_index]
    backwardreactions = [li[i] for i in range(len(li)) if i in backwardreactions_index]
    reversiblereactions = [li[i] for i in range(len(li)) if i in reversiblereactions_index]

    # Filter forward reactions
    legit_forwardreactions = [i.split(' --> ') for i in forwardreactions]
    legit_forwardreactions = [i for i in legit_forwardreactions if ((len(i[0].replace(' ', '')) > 0)&(len(i[1].replace(' ', '')) > 0))]
    
    for_reactants = [[item for item in i[0].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_forwardreactions]
    for_products = [[item for item in i[1].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_forwardreactions]
    
    forwards = list(zip(for_reactants, for_products))

    # Filter backward reactions
    legit_backwardreactions = [i.split(' <-- ') for i in backwardreactions]
    legit_backwardreactions = [i for i in legit_backwardreactions if ((len(i[0].replace(' ', '')) > 0)&(len(i[1].replace(' ', '')) > 0))]
    
    back_reactants = [[item for item in i[1].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_backwardreactions]
    back_products = [[item for item in i[0].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_backwardreactions]
    
    backwards = list(zip(back_reactants, back_products))

    # Filter reversible forward-backward reactions
    legit_reversiblereactions = [i.split(' <=> ') for i in reversiblereactions]
    legit_reversiblereactions = [i for i in legit_reversiblereactions if ((len(i[0].replace(' ', '')) > 0)&(len(i[1].replace(' ', '')) > 0))]

    rev_for_reactants = [[item for item in i[0].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_reversiblereactions]
    rev_for_products = [[item for item in i[1].split(' ') if ((item.replace('.','',1).isdigit() == False) & (item != '+'))] for i in legit_reversiblereactions]

    rev_forwards = list(zip(rev_for_reactants, rev_for_products))
    rev_backwards = list(zip(rev_for_products, rev_for_reactants))
    
    # Combine all reactions
    all_reactions = forwards + backwards + rev_forwards + rev_backwards

    # Convert reactions to edges
    react_edge = [[[(item[0][i],item[1][j]) for j in range(len(item[1]))] for i in range(len(item[0]))] for item in all_reactions]
    react_edge__ = [item for sublist in react_edge for item in sublist]
    edges = [item for sublist in react_edge__ for item in sublist]

    edgelist = list(set(edges))
    nodelist = list(set([item for sublist in edgelist for item in sublist]))

    # Create and export networkX graph
    graph = nx.from_edgelist(edgelist, create_using= nx.DiGraph())
    pd_adj = nx.to_pandas_adjacency(graph)
    pd_adj.to_csv(f'{species}_Adjacency.csv')

    # Export cleaned edgelist
    B_edgelist = [[j.replace('[', '_').replace(']', '') for j in i] for i in edgelist]
    pd.DataFrame(B_edgelist).to_csv(f'{species}_Edgelist.txt', header = None, index = None, sep = '\t')